In [1]:
import pandas as pd
import csv
import math

Создание словаря с рейтингами каждого юзера

In [2]:
def get_rating_dict(filename = "lastfm_user_scrobbles.csv"):
    with open(filename, 'r') as f:
        r = csv.reader(f)
        output = dict()
        for i, line in enumerate(r):
            if i == 0:
                continue
            user    = line[0]
            artist = line[1]
            rate    = int(line[2])
            if not user in output:
                output[user] = dict()
            output[user][artist] = rate
    return output

Реализация через pandas (оказалась медленнее)

In [3]:
def get_rating_dict_pd(df):
    output = dict()
    for i, line in df.iterrows():
        user = str(line['user_id'])
        artist = line['artist_id']
        rate    = int(line['scrobbles'])
        if not user in output:
            output[user] = dict()
        output[user][artist] = rate
    return output

Косинусная мера расстояния

In [4]:
def distCosine (vecA, vecB):
    def dotProduct (vecA, vecB):
        d = 0.0
        for dim in vecA:
            if dim in vecB:
                d += vecA[dim]*vecB[dim]
        return d
    return dotProduct (vecA,vecB) / math.sqrt(dotProduct(vecA,vecA)) / math.sqrt(dotProduct(vecB,vecB))

Поиск схожих артистов

In [5]:
def get_similar_artists(user_id, user_ratings):
    # Find like-minded users
    user_matches = [(u, distCosine(user_ratings[user_id], user_ratings[u])) for u in user_ratings if u != user_id]
    best_user_matches = sorted(user_matches, key=lambda x: x[1], reverse=True)[:1000]

    # normalization coef
    total_similarity = sum([x[1] for x in best_user_matches])
    
    # filtering closer users
    best_user_matches = dict([x for x in best_user_matches if x[1] > 0.25])

    # find top artists
    artist_similarity = dict()
    for related_user in best_user_matches:
        for artist in user_ratings[related_user]:
            if not artist in user_ratings[user_id]:
                if not artist in artist_similarity:
                    artist_similarity[artist] = 0.0
                artist_similarity[artist] += user_ratings[related_user][artist] * best_user_matches[related_user]
    
    # normalization
    for artist in artist_similarity:
        artist_similarity[artist] /= total_similarity
    
    # sort to get most relevant
    top_artists = sorted(artist_similarity.items(), key=lambda x: x[1], reverse=True)[:5]
    return [x[0] for x in top_artists]

In [6]:
df = pd.read_csv('lastfm_user_scrobbles.csv')
names_df = pd.read_csv('lastfm_artist_list.csv')

In [7]:
user_rating = get_rating_dict()

In [8]:
input_artist = 'Britney Spears'
artist_id = (names_df[names_df['artist_name']==input_artist].index + 1).tolist()
fans = df[df['artist_id']== artist_id[0]]
fans.sort_values('scrobbles', ascending=False)
top_fan = fans.iloc[0]

In [9]:
top_artists_id = get_similar_artists(str(top_fan.user_id), user_rating)

In [10]:
artists = []
for index in top_artists_id:
    temp_df = names_df[names_df['artist_id'] == int(index)]
    artists.append(temp_df['artist_name'].iloc[0])
print(artists)

['Ke$Ha', 'Shakira', 'Miley Cyrus', 'Taylor Swift', 'Paramore']
